<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/ETL_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-04 20:34:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-09-04 20:34:12 (1.21 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual+Contributions+2013+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Individual+Contributions+2013+(UF).csv"), sep=",", header=True)
df.show()

+--------+--------+-------+--------+------------------+---------+------+--------+------+----------+--------+---------+----------+-------------------+------+-----+-------+---------+---------+-----------------+-----+-----+--------------+--------------------+--------+------------------+---------------+--------+----+---------+--------+----------+--------+----------+--------+--------+--------+-------+-----+------+----------+----------+----------+----------+--------+---------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|     DATE|REFUNDDATE|               NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|             CITY|STATE|  ZIP|    OCCUPATION|             EMPNAME|EMPSTRNO|        EMPSTRNAME|        EMPCITY|EMPSTATE|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO|INTCITY|INTST|INTZIP|INTEMPNAME|INTEMPSTNO|INTEMPSTNM|INTEMPCITY|INTEMPST|INTO

In [5]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY','REFUNDDATE','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+-------+--------+------------------+---------+-------------------+------+---------+-----------------+-----+-----+--------+----+---------+--------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|     DATE|               NAME|C_CODE|BOROUGHCD|             CITY|STATE|  ZIP|EMPSTATE|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+-------+--------+------------------+---------+-------------------+------+---------+-----------------+-----+-----+--------+----+---------+--------+----------+
|    2013|       5|   1008|       P|Peterson, Daniel P|3/11/2012|   Haywood, David B|   IND|        X|            Bronx|   NY|10458|      NY|  20|       20|       0|         4|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/6/2012|      Herlihy, Neil|   IND|        Z|       Hunnington|   NY|11746|    null|  25|        0|       0|         2|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/5/2012|       Iocca, Karen|   IND|        Z|           Ho

In [6]:
# check data types
df.dtypes

[('ELECTION', 'string'),
 ('OFFICECD', 'string'),
 ('RECIPID', 'string'),
 ('CANCLASS', 'string'),
 ('RECIPNAME', 'string'),
 ('DATE', 'string'),
 ('NAME', 'string'),
 ('C_CODE', 'string'),
 ('BOROUGHCD', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIP', 'string'),
 ('EMPSTATE', 'string'),
 ('AMNT', 'string'),
 ('MATCHAMNT', 'string'),
 ('PREVAMNT', 'string'),
 ('PAY_METHOD', 'string')]

In [7]:
#Change column names
df1 = df \
.withColumnRenamed("ELECTION", "Election") \
.withColumnRenamed("RECIPID", "CandidateID") \
.withColumnRenamed("RECIPNAME", "CandidateName") \
.withColumnRenamed("DATE", "Date") \
.withColumnRenamed("NAME", "Name") \
.withColumnRenamed("C_CODE", "ContributerType") \
.withColumnRenamed("BOROUGHCD", "BoroughName") \
.withColumnRenamed("CITY", "City") \
.withColumnRenamed("STATE", "State") \
.withColumnRenamed("ZIP", "ZipCode") \
.withColumnRenamed("EMPSTATE", "ContributionState") \
.withColumnRenamed("AMNT", "Amount") \
.withColumnRenamed("MATCHAMNT", "MatchAmt") \
.withColumnRenamed("PREVAMNT", "PrevAmt") \
.withColumnRenamed("PAY_METHOD", "PayMethod") 

df1.printSchema()

root
 |-- Election: string (nullable = true)
 |-- OFFICECD: string (nullable = true)
 |-- CandidateID: string (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: string (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- MatchAmt: string (nullable = true)
 |-- PrevAmt: string (nullable = true)
 |-- PayMethod: string (nullable = true)



In [8]:
# changing datatypes of columns
df2 = df1 \
  .withColumn("Election",df1["Election"].cast("Integer")) \
  .withColumn("OFFICECD", df1["OFFICECD"].cast("Integer")) \
  .withColumn("CandidateID", df1["CandidateID"].cast("Integer")) \
  .withColumn("ZipCode", df1["ZipCode"].cast("Integer")) \
  .withColumn("Amount", df1["Amount"].cast("Float")) \
  .withColumn("MatchAmt", df1["MatchAmt"].cast("Float")) \
  .withColumn("PrevAmt", df1["PrevAmt"].cast("Float")) \
  #.withColumn("Date", df1["Date"].cast("Date")) \
  #.withColumn("REFUNDDATE", df1["REFUNDDATE"].cast("Date")) 
  
df2.printSchema()

root
 |-- Election: integer (nullable = true)
 |-- OFFICECD: integer (nullable = true)
 |-- CandidateID: integer (nullable = true)
 |-- CANCLASS: string (nullable = true)
 |-- CandidateName: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- ContributerType: string (nullable = true)
 |-- BoroughName: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- ContributionState: string (nullable = true)
 |-- Amount: float (nullable = true)
 |-- MatchAmt: float (nullable = true)
 |-- PrevAmt: float (nullable = true)
 |-- PayMethod: string (nullable = true)



In [10]:
from pyspark.sql.functions import col, unix_timestamp, to_date
df = df.withColumn('Date', 
                   to_date(unix_timestamp(col('Date'), 'MM-dd-yyyy').cast("timestamp")))
df.show()
df.printSchema()

+--------+--------+-------+--------+------------------+----+-------------------+------+---------+-----------------+-----+-----+--------+----+---------+--------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|Date|               NAME|C_CODE|BOROUGHCD|             CITY|STATE|  ZIP|EMPSTATE|AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|
+--------+--------+-------+--------+------------------+----+-------------------+------+---------+-----------------+-----+-----+--------+----+---------+--------+----------+
|    2013|       5|   1008|       P|Peterson, Daniel P|null|   Haywood, David B|   IND|        X|            Bronx|   NY|10458|      NY|  20|       20|       0|         4|
|    2013|       5|   1008|       P|Peterson, Daniel P|null|      Herlihy, Neil|   IND|        Z|       Hunnington|   NY|11746|    null|  25|        0|       0|         2|
|    2013|       5|   1008|       P|Peterson, Daniel P|null|       Iocca, Karen|   IND|        Z|           Hobken|   NJ| 7030|    null|  25

In [ ]:
#Change vaule name in ContributerType
from pyspark.sql.functions import regexp_replace
df4 = df3.withColumn('ContributerType', regexp_replace('ContributerType', 'CAN', 'Candidate')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'CORP', 'Corporation')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'EMPO', 'Labor Union')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'FAM', 'Candidate Family')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'IND', 'Individual')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMZ', 'Party Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'ORG', 'Orgainization')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'LLC', 'Limited Liability Company')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMP', 'Political Action Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'SPO', 'Spouse')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'OTHR', 'Other')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PCOMC', 'Candidate Committee')) \
  .withColumn('ContributerType', regexp_replace('ContributerType', 'PART', 'Individual')) \


In [ ]:
#Change Payment Method Name
df5=df4.withColumn('PayMethod', regexp_replace('PayMethod','0','Unknown')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','1','Cash')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','2','Check')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','3','Other')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','4','Credit Card')) \
  .withColumn('PayMethod', regexp_replace('PayMethod','5','Money Order')) \

In [ ]:
#Change Borough name
from pyspark.sql.functions import regexp_replace
df6 = df5.withColumn('BoroughName', regexp_replace('BoroughName', 'K', 'Brooklyn')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'M', 'Manhattan')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Q', 'Queens')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'S', 'Staten Island')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'X', 'Bronx')) \
  .withColumn('BoroughName', regexp_replace('BoroughName', 'Z', 'Other')) \


In [ ]:
#Call only Mayor (1) and particaptes (P) within Dataframe
df7 = df6.filter((df6.OFFICECD=="1") & (df6.CANCLASS=="P"))
df7.show()


In [ ]:
#Check the above dataframe to make sure only unique values between the OFFICECD is 1 
df6.select('OFFICECD').distinct().show()

In [ ]:
#Drop OFFICECD and CANCLASS
list2 = ['OFFICECD', 'CANCLASS'] 
df7 = df6.drop(*list2)
df7.show()

In [ ]:
# ADD AMNT and MATCHAMNT PrevAmnt
from pyspark.sql.functions import col
df8 = df7.withColumn("TotalAmount", col("Amount")+col("MatchAmt")+col('PrevAmt'))
df8.show()